# Retrieval-Augmented Generation (RAG)

## 1. Что такое RAG и для чего он нужен

**Retrieval-Augmented Generation (RAG)** - это архитектура, в которой LLM использует не только свои параметры, но и внешние данные из хранилища (обычно векторной БД) для ответа на запрос.

Основные проблемы, которые решает RAG:

1. Ограничение по дате обучения модели (knowledge cutoff).
2. Галлюцинации и вымысел.
3. Необходимость работать с внутренними, закрытыми документами.

Типичный сценарий применения:

* корпоративные ассистенты;
* поисковые системы по внутренним документам;
* юридические/медицинские справочные системы;
* аналитику по отчетам, регламентам, документации.

## 2. Общая схема работы RAG

Работу RAG обычно делят на два больших этапа:

1. **Ingestion (подготовка данных)**:
   парсинг документов --> разбиение на фрагменты (chunking) --> построение эмбеддингов --> запись во векторную БД (например, Qdrant).

2. **Online-процесс (ответ на запрос)**:
   запрос пользователя --> embedding запроса --> поиск релевантных фрагментов в Qdrant --> формирование prompt с контекстом --> генерация ответа LLM (через vLLM).

## 3. Компоненты RAG

### 3.1. Parser

Назначение: превратить сырые документы (DOCX, PDF, HTML, TXT и т.п.) в нормализованный текст плюс полезные метаданные (название файла, раздел, номер страницы и т.д.).

Ключевые задачи:

* извлечь текст;
* очистить от шума (лишние переводы строк, номера страниц, оглавления);
* сохранить структуру (заголовки, разделы), если это важно для дальнейшего chunking.

### 3.2. Chunker

**Задача чанкинга** - разбить длинный текст на фрагменты (chunks), которые:

1. Достаточно короткие, чтобы помещаться в контекст LLM.
2. Достаточно информативные, чтобы каждый chunk содержал законченную мысль.

Типичные подходы:

1. **Фиксированный размер по токенам/символам**
   Простой вариант: брать, например, 500 токенов с overlap 100 токенов.
   Плюсы: простота реализации.
   Минусы: разрыв логики (предложение может резаться посередине), слабая семантика.

2. **Чанк по структуре документа**
   Разбиение по заголовкам, абзацам, спискам.
   Плюсы: chunks ближе к логическим блокам документа.
   Минусы: разный размер фрагментов, иногда очень длинные разделы.

3. **Semantic chunking**
   Используются эвристики или модели (например, кластеризация предложений) для объединения предложений в семантически однородные блоки.
   Плюсы: лучшие результаты при поиске.
   Минусы: сложнее реализация, больше вычислений.

### 3.3. Embedder

**Embedder** - модель, которая переводит текст в векторы фиксированной размерности (эмбеддинги).

Назначение:

* Для каждого chunk вычислить embedding и записать его в векторную БД.
* Для каждого пользовательского запроса - также получить embedding и использовать его для поиска.

Типы моделей:

* Специализированные embedding-модели: BGE-M3, E5, Instructor, GTE и т.д.
* Embedding-эндпоинты облачных провайдеров (OpenAI, Azure, и пр.).

Ключевые характеристики эмбеддера:

* **Размерность вектора** (например, 768, 1024, 1536). Она должна совпадать с размерностью коллекции в Qdrant.
* **Тип задачи**:

- **text embedding** (для query + passage)
- **multi‑lingual embedding**
- **cross‑encoder vs bi‑encoder** (для reranking)

Рабочий цикл:

1. На этапе ingestion:
   chunk --> embedder --> вектор --> запись в Qdrant.
2. На этапе ответа:
   query --> embedder --> вектор --> поиск в Qdrant --> контекст.

## 4. Qdrant: векторная БД

Qdrant - это векторная база данных, которая хранит векторы и метаданные и предоставляет API для:

* создания и управления коллекциями;
* записи и обновления точек (points);
* поиска ближайших векторов;
* фильтрации по метаданным;
* шардирования и репликации.

### 4.1. Базовые сущности

* **Collection** - логическое хранилище векторов и метаданных.
* **Point** - объект с полями:

  * `id` - уникальный идентификатор;
  * `vector` - embedding;
  * `payload` - произвольные метаданные (текст chunk-а, source, тип документа и т.д.).


### 4.2. Основные методы Qdrant

С точки зрения RAG-important операций:

1. **Создание коллекции**

   * `create_collection`
     Определяется:

     * размерность вектора (`size`);
     * метрика (`distance` - cosine, dot, euclid);
     * конфигурация HNSW, sharding, количество реплик.

2. **Информация о коллекции**

   * `get_collection_info`
     Возвращает конфигурацию и состояние коллекции (размер, количество точек, конфиг индекса и т.д.).

3. **Запись данных**

   * `upsert`
     Добавление или обновление точек.
     Используется в ingestion-пайплайне.
     Принимает список `PointStruct` c `id`, `vector`, `payload`.

   * `delete`
     Удаление точек по id или по фильтру.

4. **Поиск**

   * `search`
     Классический nearest-neighbor поиск по одному вектору (query_vector).
     Параметры:

     * `collection_name`
     * `query_vector`
     * `limit` (top-K)
     * `filter` (по payload: например, по типу документа, дате, языку)
     * `with_payload`, `with_vectors`

   * `query_points` (в новых версиях)
     Расширенный запрос, который поддерживает:

     * поиск по нескольким векторным полям;
     * скоры;
     * дополнительные настройки.

   * `scroll`
     «Прокрутка» по коллекции: постраничный проход через все `points` (для отладки, миграций, бэкапов).

5. **Управление индексами**

   * `create_field_index`, `delete_field_index`
     Индексы по payload-полям для ускорения фильтрации.

6. **Обновление конфигурации**

   * `update_collection`, `update_cluster` и др.
     Настройки шардирования, репликации, оптимизации.


### 4.3. Режимы и типы поиска в Qdrant

1. **Vector similarity search**

   * Поиск ближайших векторов по метрике (cosine / dot / euclid).
     Плюсы: работает с высоким семантическим сходством;
     Минусы: иногда возвращает семантически схожие, но не релевантные конкретному вопросу chunks.

2. **Hybrid search (vector + keyword)**

   Можно комбинировать:

   * Vector search (по embedding)
   * Keyword/BM25 (по тексту или по payload)

   Обычно реализуется как последовательное или комбинированное:

   * сначала BM25 фильтрует кандидатов --> затем vector reranking;
   * либо vector search + фильтр по полям.

   Плюсы: лучше работает для специфических терминов, кодов, чисел;
   Минусы: сложнее организовать, нужна дополнительная инфраструктура (например, отдельный text index).

3. **Filtered search**

   В `filter` можно указать условия по payload, например:

   * тип документа;
   * дата;
   * язык;
   * определенные теги.

   Плюсы: позволяет сузить поиск до пригодного подмножества данных (например, только документы определенного проекта).
   Минусы: требует грамотного проектирования payload-структуры.

4. **Multi-vector / multi-field search**

   Qdrant поддерживает коллекции с несколькими векторными полями (например, `content_vector`, `title_vector`).
   Поиск может учитывать оба поля, с разными весами.

   Плюсы: можно сочетать разные типы эмбеддингов;
   Минусы: сложнее настройка и балансировка.

## 5. vLLM: сервер инференса LLM

vLLM - это high-performance сервер инференса LLM с упором на:

* эффективное управление KV-кэшем (PagedAttention);
* высокую пропускную способность по токенам;
* OpenAI-совместимый API.

Используется как backend для:

* генерации ответов RAG;
* reranking;
* query rewriting;
* summarization.

### 5.1. Запуск vLLM

Базовый запуск:

```bash
vllm serve your-model-name \
  --port 8000 \
  --host 0.0.0.0
```

```bash
vllm  | INFO 12-04 08:26:12 [api_server.py:1090] Starting vLLM API server on http://0.0.0.0:8000
vllm  | INFO 12-04 08:26:12 [launcher.py:28] Available routes are:
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /openapi.json, Methods: HEAD, GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /docs, Methods: HEAD, GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /docs/oauth2-redirect, Methods: HEAD, GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /redoc, Methods: HEAD, GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /health, Methods: GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /load, Methods: GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /ping, Methods: GET, POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /tokenize, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /detokenize, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/models, Methods: GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /version, Methods: GET
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/chat/completions, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/completions, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/embeddings, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /pooling, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /score, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/score, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/audio/transcriptions, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /rerank, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v1/rerank, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /v2/rerank, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /invocations, Methods: POST
vllm  | INFO 12-04 08:26:12 [launcher.py:36] Route: /metrics, Methods: GET
vllm  | INFO:     Started server process [1]
vllm  | INFO:     Waiting for application startup.
vllm  | INFO:     Application startup complete.
vllm  | INFO:     172.19.0.1:36838 - "GET /health HTTP/1.1" 200 OK
```


### 5.2. Важные флаги vLLM

1. **Модель и пути**

   * `--model`
     Название модели из HuggingFace или локальный путь.

   * `--download-dir`
     Каталог, куда качать веса. Позволяет кешировать модели между перезапусками.

2. **Аппаратные ресурсы**

   * `--tensor-parallel-size`
     Количество GPU, используемых для tensor parallelism.

   * `--dtype`
     Тип чисел: `float16`, `bfloat16`, иногда `float32`.
     Важен баланс между точностью и памятью.

   * `--max-num-batched-tokens`
     Ограничение на общее число токенов в батче (сумма по всем запросам).
     Влияет на throughput и стабильность.

   * `--max-num-seqs`
     Максимальное число последовательностей (запросов) в батче.

   * `--gpu-memory-utilization`
     Целевой процент использования GPU памяти (например, 0.9). vLLM подгоняет под это размер кэша и другие параметры.

3. **Контекст и кэш**

   * `--max-context-len` или `--max-model-len`
     Максимальная длина контекста (prompt + generated tokens).
     Чем больше, тем выше требования к памяти.

   * Параметры, связанные с PagedAttention и кэшированием KV (в разных версиях - разные флаги), определяют, сколько токенов истории можно держать и как эффективно переиспользовать их между запросами.

4. **Сервер и API**

   * `--port`

   * `--host`

   * `--served-model-name`
     Удобно, если нужно публиковать модель под другим именем, чем на HF.

   * `--enable-metrics`
     Экспорт метрик (обычно в формате Prometheus).

   * `--log-level`
     Для отладки и мониторинга.

5. **Batching и планировщик**

   vLLM автоматически выполняет smart batching запросов.
   Но параметры `--max-num-batched-tokens`, `--max-num-seqs` и возможные флаги, связанные с временем ожидания batch (например, max batch latency), сильно влияют на latency vs throughput.

   Настройки аналогичны Triton:

   * При высоком потоке запросов есть смысл позволить больший batch, чтобы повысить throughput.
   * Для интерактивных сценариев иногда нужно уменьшать batch или ограничивать задержку формирования batch.


### 5.3. Использование vLLM как OpenAI-совместимого API

После запуска можно использовать такой код:

```python
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="dummy",
)

resp = client.chat.completions.create(
    model="your-model-name",
    messages=[
        {"role": "system", "content": "Ты помощник."},
        {"role": "user", "content": "Объясни, что такое RAG."},
    ],
)
```

Это удобно, потому что:

* не нужно менять клиентский код при переходе от внешнего OpenAI к локальному vLLM;
* можно использовать те же структуры запросов (chat/completions).

## 6. Способы улучшить базовый RAG

Базовая схема: query --> embedder --> Qdrant --> контекст --> LLM.
Эту схему можно существенно улучшить за счет дополнительных компонент.

### 6.1. Query Rewriter

Модуль, который переписывает запрос перед retrieval:

* убирает лишние детали и шум;
* добавляет недостающий контекст;
* разбивает сложный вопрос на несколько подзапросов;
* переводит на язык документов (например, формализует запрос под официальную терминологию).

Реализация:

* отдельная LLM (через vLLM) с промптом вида:
  «Перепиши запрос пользователя так, чтобы он лучше подходил для поиска по базе документов…».

Эффект: увеличивается recall релевантных документов.


### 6.2. Reranker (Cross-Encoder / LLM-Reranker)

После начального поиска по эмбеддингам можно прогнать top-N документов через reranker:

1. На первом этапе Qdrant возвращает, например, 20–50 кандидатов по cosine similarity.
2. Reranker оценивает парой (query, chunk) и выдает более точный «релевантность score».
3. Берутся top-K после reranking (например, 5–10).

Варианты:

* готовые модели reranking (например, cross-encoder с HuggingFace);
* LLM, который действует как reranker: на вход подается query и несколько кандидатов, на выход - ранжированный список.

Плюсы:

* значительно снижает число нерелевантных chunk-ов в контексте.

Минусы:

* дополнительная нагрузка (дополнительные вызовы модели).


### 6.3. Multi-step / Decomposed RAG

Используется для сложных запросов:

1. LLM анализирует вопрос и разлагает его на подзадачи.
2. Для каждой подзадачи выполняется retrieval и частичное решение.
3. Финальный ответ собирается из нескольких шагов reasoning.

Примеры техник:

* Chain-of-Thought RAG;
* Tree-of-Thought RAG;
* Tool-augmented RAG (LLM как оркестратор инструментов).


### 6.4. Context Optimization

Несколько техник:

* **Контекстное сжатие (context compression)** - длинные найденные chunk-и сначала суммируются/сжимаются, затем подаются в LLM.
* **Adaptive context window** - динамический выбор количества chunk-ов в зависимости от сложности вопроса.
* **Metadata-aware retrieval** - использование фильтров по метаданным для уменьшения шума.


### 6.5. Knowledge Graph / GraphRAG

При наличии сложной структуры знаний:

* выделяются сущности и связи;
* строится граф (knowledge graph);
* при запросе выполняется traversal по графу и выбор релевантных узлов;
* затем результаты передаются в LLM.

Плюсы: хорошо для аналитических запросов, когда важна структура и причинно-следственные связи.
Минусы: сложность обработки и поддержки.